<a href="https://colab.research.google.com/github/kaifkh20/nn_z_to_h/blob/main/makemore_a_z.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving names(1).txt to names(1).txt


In [2]:
words = open('names(1).txt').read().splitlines()

In [3]:
import torch

In [4]:
N = torch.zeros((27,27),dtype=torch.int32)

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [6]:
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]

    N[ix1,ix2] +=1


In [7]:
N[0,:]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

p = N[ix].float()
    p/=p.sum()
this is inneficient

what is efficient is :
P/=P.sum(1,keepdim=true)  

Model smoothing instead of assigning not encountered pair of words as 0 we make it 1

In [8]:
P = (N+1).float()
P/=P.sum(1,keepdim=True)

Multinomial function is draw samples from probability distribution

In [9]:
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
  ix = 0
  out = []
  while True:
    p = P[ix]
    ix = torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
    out.append(itos[ix])
    if ix==0:
      break


  print(''.join(out))

junide.
janasah.
p.
cony.
a.


import torch

P = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])

# Sum across rows (axis 0) without keepdim
row_sums = P.sum(0)  # Output: tensor([5, 7, 9])

# Sum across columns (axis 1) without keepdim
column_sums = P.sum(1)  # Output: tensor([ 6, 15])

# Sum across rows (axis 0)
row_sums = P.sum(0, keepdim=True)  # Output: tensor([[5, 7, 9]])

# Sum across columns (axis 1)
column_sums = P.sum(1, keepdim=True)  # Output: tensor([[ 6],
                                             #          [15]])

In [10]:
log_likelihood=0.0
n=0
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1,ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n+=1

nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n=}')

nll=tensor(559951.5625)
nll/n=tensor(2.4544)


Create training set of bigram (x,y)

In [11]:
xs,ys = [],[]
for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys) #the correct y

xs.shape,xs

(torch.Size([5]), tensor([ 0,  5, 13, 13,  1]))

In [ ]:
import torch.nn.functional as F
xenc = F.one_hot(xs,num_classes=27).float()
xenc,xenc.shape

5 (Input Layer) -> 27 neuron (Second Layer)

In [25]:
#randomly initialize 27 Weights for each neurons(27)
g = torch.Generator().manual_seed(2147483647)
W = torch.randn([27,27],generator=g,requires_grad = True)

In [32]:
for k in range(100):
  xenc = F.one_hot(xs,num_classes=27).float() #input to nnet
  logits = xenc @ W #predict log-counts
  counts = logits.exp() #counts equivalent to N
  prob = counts/counts.sum(1,keepdims=True) #prob for next char (softmax)
  loss = -prob[torch.arange(5),ys].log().mean() + 1*(W**2).mean() #negative log-likelihood + regularization term

  print(loss.item())

  W.grad = None #set it to zero
  loss.backward()

  W.data += -0.1 * W.grad #update

2.035191774368286
2.02927827835083
2.023423671722412
2.017627000808716
2.011887788772583
2.0062057971954346
2.000580072402954
1.9950106143951416
1.9894965887069702
1.984037160873413
1.9786324501037598
1.9732813835144043
1.9679837226867676
1.9627389907836914
1.9575464725494385
1.9524058103561401
1.9473164081573486
1.9422777891159058
1.9372892379760742
1.9323503971099854
1.927460789680481
1.9226198196411133
1.9178273677825928
1.913082242012024
1.9083845615386963
1.903733253479004
1.8991281986236572
1.8945691585540771
1.8900549411773682
1.8855857849121094
1.8811602592468262
1.8767790794372559
1.8724408149719238
1.8681455850601196
1.8638924360275269
1.8596813678741455
1.8555113077163696
1.8513824939727783
1.8472940921783447
1.843245506286621
1.8392366170883179
1.8352668285369873
1.8313356637954712
1.8274425268173218
1.823587417602539
1.8197696208953857
1.815988540649414
1.8122440576553345
1.8085355758666992
1.8048629760742188
1.801225185394287
1.7976226806640625
1.7940540313720703
1.790519

In [33]:
prob

tensor([[0.0271, 0.0059, 0.0072, 0.0026, 0.0095, 0.6126, 0.0017, 0.0126, 0.0079,
         0.0162, 0.0048, 0.0147, 0.0055, 0.0050, 0.0235, 0.0591, 0.0270, 0.0016,
         0.0134, 0.0034, 0.0173, 0.0064, 0.0018, 0.0110, 0.0069, 0.0483, 0.0470],
        [0.0129, 0.0282, 0.0113, 0.0207, 0.0483, 0.0128, 0.0047, 0.0145, 0.0048,
         0.0133, 0.0258, 0.0105, 0.0056, 0.6310, 0.0054, 0.0138, 0.0129, 0.0024,
         0.0310, 0.0099, 0.0196, 0.0132, 0.0201, 0.0014, 0.0058, 0.0012, 0.0192],
        [0.0101, 0.3875, 0.0139, 0.0106, 0.0172, 0.0071, 0.0088, 0.0084, 0.0239,
         0.0060, 0.0030, 0.0189, 0.0049, 0.3660, 0.0054, 0.0043, 0.0157, 0.0046,
         0.0168, 0.0024, 0.0073, 0.0031, 0.0050, 0.0072, 0.0237, 0.0136, 0.0048],
        [0.0101, 0.3875, 0.0139, 0.0106, 0.0172, 0.0071, 0.0088, 0.0084, 0.0239,
         0.0060, 0.0030, 0.0189, 0.0049, 0.3660, 0.0054, 0.0043, 0.0157, 0.0046,
         0.0168, 0.0024, 0.0073, 0.0031, 0.0050, 0.0072, 0.0237, 0.0136, 0.0048],
        [0.6119, 0.0045,